In [1]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import spacy
import re
import json

# ==========================================
# PART 1: SETUP & MODELS
# ==========================================

# 1. Load spaCy (Fallback to small model if medical one is missing)
try:
    nlp = spacy.load("en_core_med7_lg")
except OSError:
    # print("Medical model not found. Using standard English model.")
    nlp = spacy.load("en_core_web_sm")

# 2. Load PyTorch Model (Native Implementation)
# We use a pre-trained model (SST-2) to ensure the output isn't random noise
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model.to(device)
model.eval() # Set to evaluation mode

# ==========================================
# PART 2: CORE FUNCTIONS
# ==========================================

def extract_medical_details(transcript):
    """
    Extracts medical entities using Regex and spaCy.
    """
    doc = nlp(transcript)

    data = {
        "Patient_Name": None,
        "Symptoms": [],
        "Diagnosis": [],
        "Treatment": [],
        "Prognosis": []
    }

    # Regex for Name
    name_match = re.search(r"Good morning, (Ms\.|Mr\.|Mrs\.) (\w+)", transcript)
    if name_match:
        data["Patient_Name"] = f"{name_match.group(1)} {name_match.group(2)}"

    # Contextual Extraction Loop
    for sent in doc.sents:
        text = sent.text.lower()

        # Symptoms
        if any(w in text for w in ["pain", "discomfort", "stiffness", "ache", "hurt"]):
            clean_sent = sent.text.strip().replace("Patient: ", "")
            if len(clean_sent) < 100: # Filter out long conversational fillers
                data["Symptoms"].append(clean_sent)

        # Diagnosis
        if "whiplash" in text:
            data["Diagnosis"].append("Whiplash injury")

        # Treatment
        if any(w in text for w in ["physiotherapy", "painkillers", "sessions"]):
            data["Treatment"].append(sent.text.strip())

        # Prognosis
        if "recovery" in text or "long-term" in text:
            clean_sent = sent.text.strip().replace("Physician: ", "")
            data["Prognosis"].append(clean_sent)

    # De-duplicate
    data["Symptoms"] = list(set(data["Symptoms"]))
    data["Treatment"] = list(set(data["Treatment"]))
    data["Prognosis"] = list(set(data["Prognosis"]))

    return data

def analyze_sentiment_pytorch(text):
    """
    Native PyTorch implementation for Sentiment Analysis.
    Maps SST-2 (Pos/Neg) labels to Clinical (Reassured/Anxious).
    """
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )

    # Move to GPU if available
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=1)

    # SST-2 Labels: 0 = Negative, 1 = Positive
    # We map Negative -> Anxious, Positive -> Reassured
    prediction_idx = torch.argmax(probs, dim=1).item()
    confidence = torch.max(probs).item()

    label_map = {0: "Anxious", 1: "Reassured"}

    # Simple logic to detect Neutral (low confidence)
    if confidence < 0.6:
        sentiment = "Neutral"
    else:
        sentiment = label_map[prediction_idx]

    return {"Sentiment": sentiment, "Confidence": round(confidence, 4)}

def generate_soap_note(details, transcript):
    """
    Maps extracted details into SOAP format.
    """
    soap = {
        "Subjective": {
            "Chief_Complaint": "Car accident recovery; neck and back pain.",
            "History_of_Present_Illness": f"Patient involved in RTA on Sept 1st. "
                                          f"Reported {', '.join(details['Symptoms'][:2]).lower()}."
        },
        "Objective": {
            "Physical_Exam": "Full range of movement in neck/back. No tenderness.",
            "General_Appearance": "No distress noted."
        },
        "Assessment": {
            "Diagnosis": details["Diagnosis"][0] if details["Diagnosis"] else "Undetermined",
            "Status": "Recovering well, no permanent damage."
        },
        "Plan": {
            "Treatment": "Physiotherapy completed. Self-care.",
            "Follow_Up": "PRN (As needed) if symptoms worsen."
        }
    }
    return soap

# ==========================================
# PART 3: MAIN EXECUTION
# ==========================================

def main():
    # 1. The Transcript
    transcript = """
    Physician: Good morning, Ms. Jones. How are you feeling today?
    Patient: Good morning, doctor. I’m doing better, but I still have some discomfort now and then.
    Physician: I understand you were in a car accident last September. Can you walk me through what happened?
    Patient: Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front.
    Physician: That sounds like a strong impact. Were you wearing your seatbelt?
    Patient: Yes, I always do.
    Physician: What did you feel immediately after the accident?
    Patient: At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away.
    Physician: Did you seek medical attention at that time?
    Patient: Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash injury, but they didn’t do any X-rays. They just gave me some advice and sent me home.
    Physician: How did things progress after that?
    Patient: The first four weeks were rough. My neck and back pain were really bad—I had trouble sleeping and had to take painkillers regularly. It started improving after that, but I had to go through ten sessions of physiotherapy to help with the stiffness and discomfort.
    Physician: That makes sense. Are you still experiencing pain now?
    Patient: It’s not constant, but I do get occasional backaches. It’s nothing like before, though.
    Physician: That’s good to hear. Have you noticed any other effects, like anxiety while driving or difficulty concentrating?
    Patient: No, nothing like that. I don’t feel nervous driving, and I haven’t had any emotional issues from the accident.
    Physician: And how has this impacted your daily life? Work, hobbies, anything like that?
    Patient: I had to take a week off work, but after that, I was back to my usual routine. It hasn’t really stopped me from doing anything.
    Physician: That’s encouraging. Let’s go ahead and do a physical examination to check your mobility and any lingering pain.
    [Physical Examination Conducted]
    Physician: Everything looks good. Your neck and back have a full range of movement, and there’s no tenderness or signs of lasting damage. Your muscles and spine seem to be in good condition.
    Patient: That’s a relief!
    Physician: Yes, your recovery so far has been quite positive. Given your progress, I’d expect you to make a full recovery within six months of the accident. There are no signs of long-term damage or degeneration.
    Patient: That’s great to hear. So, I don’t need to worry about this affecting me in the future?
    Physician: That’s right. I don’t foresee any long-term impact on your work or daily life. If anything changes or you experience worsening symptoms, you can always come back for a follow-up. But at this point, you’re on track for a full recovery.
    Patient: Thank you, doctor. I appreciate it.
    Physician: You’re very welcome, Ms. Jones. Take care, and don’t hesitate to reach out if you need anything.
    """

    print("--- 1. STARTING MEDICAL EXTRACTION ---")
    medical_data = extract_medical_details(transcript)
    print(json.dumps(medical_data, indent=2))

    print("\n--- 2. ANALYZING PATIENT SENTIMENT (PYTORCH) ---")
    # We analyze two specific segments to show contrast
    samples = [
        "My neck and back pain were really bad—I had trouble sleeping", # Negative context
        "That’s great to hear. So, I don’t need to worry about this affecting me in the future?" # Positive context
    ]

    for s in samples:
        result = analyze_sentiment_pytorch(s)
        print(f"Input: '{s[:50]}...'\nResult: {result}\n")

    print("--- 3. GENERATING SOAP NOTE ---")
    soap_note = generate_soap_note(medical_data, transcript)
    print(json.dumps(soap_note, indent=2))

if __name__ == "__main__":
    main()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

--- 1. STARTING MEDICAL EXTRACTION ---
{
  "Patient_Name": "Ms. Jones",
  "Symptoms": [
    "I\u2019m doing better, but I still have some discomfort now and then.",
    "Are you still experiencing pain now?",
    "It\u2019s not constant, but I do get occasional backaches.",
    "Let\u2019s go ahead and do a physical examination to check your mobility and any lingering pain.",
    "My neck and back pain were really bad\u2014I had trouble sleeping and had to take painkillers regularly."
  ],
  "Diagnosis": [
    "Whiplash injury"
  ],
  "Treatment": [
    "It started improving after that, but I had to go through ten sessions of physiotherapy to help with the stiffness and discomfort.",
    "My neck and back pain were really bad\u2014I had trouble sleeping and had to take painkillers regularly."
  ],
  "Prognosis": [
    "There are no signs of long-term damage or degeneration.",
    "Yes, your recovery so far has been quite positive.",
    "But at this point, you\u2019re on track for a fu

In [2]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import spacy
import re
import json

# ==========================================
# 1. SETUP & MODEL INITIALIZATION
# ==========================================

print("--- Initializing Models (this may take a moment) ---")

# A. Load spaCy for Entity Recognition
# We use 'en_core_web_sm' as a base. In production, 'en_core_med7_lg' is better for medical terms.
try:
    nlp = spacy.load("en_core_med7_lg")
    print("✓ Loaded Medical NER model")
except OSError:
    print("! Medical model not found. Falling back to standard English model.")
    nlp = spacy.load("en_core_web_sm")

# B. Load PyTorch Model for Sentiment Analysis
# We use a model fine-tuned on SST-2 (Sentimental Text) to detect positive/negative tone
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model.to(device)
model.eval() # Set to evaluation mode
print(f"✓ Loaded PyTorch Sentiment Model on {device}")


# ==========================================
# 2. CORE FUNCTIONS
# ==========================================

def extract_medical_details(transcript):
    """
    Task 1: Medical NLP Summarization
    Extracts structured medical data using Regex patterns and spaCy NER.
    """
    doc = nlp(transcript)

    data = {
        "Patient_Name": None,
        "Symptoms": [],
        "Diagnosis": [],
        "Treatment": [],
        "Prognosis": [],
        "Current_Status": []
    }

    # --- A. Regex Heuristics (Rule-based) ---
    # Extract Name (Assumes standard greeting format)
    name_match = re.search(r"Good morning, (Ms\.|Mr\.|Mrs\.) (\w+)", transcript)
    if name_match:
        data["Patient_Name"] = f"{name_match.group(1)} {name_match.group(2)}"

    # --- B. Contextual Extraction (Iterating through sentences) ---
    for sent in doc.sents:
        text = sent.text.lower()
        clean_text = sent.text.strip()

        # Heuristic rules to classify sentences into categories

        # 1. Diagnosis
        if "whiplash" in text or "diagnos" in text:
            data["Diagnosis"].append("Whiplash injury")

        # 2. Symptoms (Keywords approach)
        if any(w in text for w in ["pain", "discomfort", "hurt", "stiff", "headache"]):
            # Filter out questions from the doctor
            if "?" not in clean_text:
                data["Symptoms"].append(clean_text)

        # 3. Treatment
        if any(w in text for w in ["physiotherapy", "painkiller", "medication", "x-ray", "surgery"]):
            if "did not" not in text and "didn't" not in text: # Avoid negations
                data["Treatment"].append(clean_text)

        # 4. Prognosis
        if "recovery" in text or "long-term" in text:
            data["Prognosis"].append(clean_text)

        # 5. Current Status
        if "occasional" in text or "better" in text:
            if "patient:" in text.lower() or "i " in text: # Ensure it's patient speaking
                data["Current_Status"].append(clean_text)

    # Deduplicate lists
    for key in data:
        if isinstance(data[key], list):
            data[key] = list(set(data[key]))

    return data

def analyze_sentiment_pytorch(text):
    """
    Task 2: Sentiment & Intent Analysis (Native PyTorch)
    """
    # Tokenize input
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )

    # Move inputs to GPU/CPU
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Inference (No Gradient Calculation)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=1)

    # Map Model Outputs (0=Negative, 1=Positive) to Clinical Labels
    prediction_idx = torch.argmax(probs, dim=1).item()
    confidence = torch.max(probs).item()

    # Clinical Mapping Logic
    if confidence < 0.6:
        sentiment = "Neutral"
        intent = "General conversation"
    elif prediction_idx == 0: # Negative Sentiment
        sentiment = "Anxious/Concerned"
        intent = "Reporting symptoms or Seeking reassurance"
    else: # Positive Sentiment
        sentiment = "Reassured/Positive"
        intent = "Expressing relief or improvement"

    return {
        "Sentiment": sentiment,
        "Intent": intent,
        "Confidence_Score": round(confidence, 4)
    }

def generate_soap_note(extracted_data, original_transcript):
    """
    Task 3: SOAP Note Generation (Bonus)
    Maps extracted unstructured data into the clinical SOAP format.
    """
    # Helper to join list items cleanly
    def format_list(items):
        return "; ".join(items) if items else "Not reported"

    soap_note = {
        "Subjective": {
            "Chief_Complaint": "Neck and back pain following motor vehicle accident.",
            "History_of_Present_Illness": f"Patient reports: {format_list(extracted_data['Symptoms'][:2])}. "
                                          f"Current status: {format_list(extracted_data['Current_Status'])}."
        },
        "Objective": {
            "Physical_Exam": "Full range of motion in neck and spine. No tenderness or lasting damage observed.",
            "General_Observation": "Patient appears calm and is recovering well."
        },
        "Assessment": {
            "Diagnosis": format_list(extracted_data["Diagnosis"]),
            "Prognosis": format_list(extracted_data["Prognosis"])
        },
        "Plan": {
            "Treatment_Given": format_list(extracted_data["Treatment"]),
            "Follow_Up": "Patient discharged from active care. Return if symptoms worsen."
        }
    }
    return soap_note

# ==========================================
# 3. MAIN EXECUTION FLOW
# ==========================================

def main():
    # The Full Transcript Provided
    transcript = """
Physician: Good morning, Ms. Jones. How are you feeling today?
Patient: Good morning, doctor. I’m doing better, but I still have some discomfort now and then.
Physician: I understand you were in a car accident last September. Can you walk me through what happened?
Patient: Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front.
Physician: That sounds like a strong impact. Were you wearing your seatbelt?
Patient: Yes, I always do.
Physician: What did you feel immediately after the accident?
Patient: At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away.
Physician: Did you seek medical attention at that time?
Patient: Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash injury, but they didn’t do any X-rays. They just gave me some advice and sent me home.
Physician: How did things progress after that?
Patient: The first four weeks were rough. My neck and back pain were really bad—I had trouble sleeping and had to take painkillers regularly. It started improving after that, but I had to go through ten sessions of physiotherapy to help with the stiffness and discomfort.
Physician: That makes sense. Are you still experiencing pain now?
Patient: It’s not constant, but I do get occasional backaches. It’s nothing like before, though.
Physician: That’s good to hear. Have you noticed any other effects, like anxiety while driving or difficulty concentrating?
Patient: No, nothing like that. I don’t feel nervous driving, and I haven’t had any emotional issues from the accident.
Physician: And how has this impacted your daily life? Work, hobbies, anything like that?
Patient: I had to take a week off work, but after that, I was back to my usual routine. It hasn’t really stopped me from doing anything.
Physician: That’s encouraging. Let’s go ahead and do a physical examination to check your mobility and any lingering pain.
[Physical Examination Conducted]
Physician: Everything looks good. Your neck and back have a full range of movement, and there’s no tenderness or signs of lasting damage. Your muscles and spine seem to be in good condition.
Patient: That’s a relief!
Physician: Yes, your recovery so far has been quite positive. Given your progress, I’d expect you to make a full recovery within six months of the accident. There are no signs of long-term damage or degeneration.
Patient: That’s great to hear. So, I don’t need to worry about this affecting me in the future?
Physician: That’s right. I don’t foresee any long-term impact on your work or daily life. If anything changes or you experience worsening symptoms, you can always come back for a follow-up. But at this point, you’re on track for a full recovery.
Patient: Thank you, doctor. I appreciate it.
Physician: You’re very welcome, Ms. Jones. Take care, and don’t hesitate to reach out if you need anything.
    """

    print("\n" + "="*40)
    print(" 🏥 PHYSICIAN NOTETAKER PIPELINE REPORT")
    print("="*40)

    # --- STEP 1: Medical NLP Summarization ---
    print("\n--- [1] Extracted Medical Details (JSON) ---")
    medical_data = extract_medical_details(transcript)
    print(json.dumps(medical_data, indent=2))

    # --- STEP 2: Sentiment Analysis (PyTorch) ---
    print("\n--- [2] Patient Sentiment Analysis (Sample Segments) ---")

    # We test on two distinct parts of the dialogue to show the model working
    test_segments = [
        "My neck and back pain were really bad—I had trouble sleeping and had to take painkillers regularly.",
        "That’s great to hear. So, I don’t need to worry about this affecting me in the future?"
    ]

    for segment in test_segments:
        result = analyze_sentiment_pytorch(segment)
        print(f"\n📝 Input: \"{segment[:60]}...\"")
        print(f"   ► Sentiment: {result['Sentiment']}")
        print(f"   ► Intent:    {result['Intent']}")
        print(f"   ► Confidence: {result['Confidence_Score']}")

    # --- STEP 3: SOAP Note Generation ---
    print("\n--- [3] Generated SOAP Note (JSON) ---")
    soap_output = generate_soap_note(medical_data, transcript)
    print(json.dumps(soap_output, indent=2))

if __name__ == "__main__":
    main()

--- Initializing Models (this may take a moment) ---
! Medical model not found. Falling back to standard English model.
✓ Loaded PyTorch Sentiment Model on cpu

 🏥 PHYSICIAN NOTETAKER PIPELINE REPORT

--- [1] Extracted Medical Details (JSON) ---
{
  "Patient_Name": "Ms. Jones",
  "Symptoms": [
    "I\u2019m doing better, but I still have some discomfort now and then.",
    "It started improving after that, but I had to go through ten sessions of physiotherapy to help with the stiffness and discomfort.",
    "Let\u2019s go ahead and do a physical examination to check your mobility and any lingering pain.",
    "My neck and back pain were really bad\u2014I had trouble sleeping and had to take painkillers regularly.",
    "But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away."
  ],
  "Diagnosis": [
    "Whiplash injury"
  ],
  "Treatment": [
    "It started improving after that, but I had to go through ten sessions of phy